In [30]:
import os
import pandas as pd
import seaborn as sns
import json
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import logging
import transformers
from bert_score import score
import scipy

In [10]:
data_path ='outputs'
file_name = 'T0_3B_generation_t1_rp1.5_lp1_ml30_nb10_10'
file_path = os.path.join(data_path,file_name,'mrpc-negative.json')
print('Openinng',file_path)
if os.path.exists(file_path):
    with open(file_path,'r') as file:
        file = json.load(file)
    print('Accuracy',file['accuracy'])
    print('Consistency',file['consistency'])

Openinng outputs/T0_3B_generation_t1_rp1.5_lp1_ml30_nb10_10/mrpc-negative.json
Accuracy 0.6151960784313726
Consistency 0.9558823529411765


## Study of the uncertainty

In [29]:
for index, (pos,neg) in enumerate(zip(file['std answer'],file['neg answer'])):
    print(pos,neg)
    if index == 50:
        break

[0.32787883281707764, 0.6721211075782776] [0.3509794771671295, 0.6490204930305481]
[0.4285570979118347, 0.5714429616928101] [0.4165370464324951, 0.5834629535675049]
[0.6715832948684692, 0.3284166753292084] [0.6358197927474976, 0.36418023705482483]
[0.5882970094680786, 0.4117029905319214] [0.5672550201416016, 0.43274497985839844]
[0.5676711797714233, 0.4323287606239319] [0.5458579063415527, 0.45414209365844727]
[0.6119723320007324, 0.38802772760391235] [0.5975478291511536, 0.4024522006511688]
[0.5634849071502686, 0.43651506304740906] [0.5588682889938354, 0.44113171100616455]
[0.5393760800361633, 0.46062391996383667] [0.5389453172683716, 0.4610547125339508]
[0.6748654842376709, 0.3251345753669739] [0.681418776512146, 0.3185811936855316]
[0.5086709260940552, 0.4913291335105896] [0.4899972379207611, 0.5100026726722717]
[0.43567830324172974, 0.5643216967582703] [0.40939006209373474, 0.5906099677085876]
[0.22900334000587463, 0.7709965705871582] [0.2173764556646347, 0.7826235294342041]
[0.757

In [32]:
print('Pearson')
x = [i[0] for i in file['std answer']]
y = [i[0] for i in file['std answer']]
print('Pos',scipy.stats.pearsonr(x, y))

x = [i[1] for i in file['std answer']]
y = [i[1] for i in file['std answer']]
print('Neg',scipy.stats.pearsonr(x, y))

x = [i[0] for i in file['std answer']]
y = [i[1] for i in file['std answer']]
print('Pos/Neg',scipy.stats.pearsonr(x, y))

x = [i[1] for i in file['std answer']]
y = [i[0] for i in file['std answer']]
print('Neg/Pos',scipy.stats.pearsonr(x, y))

Pearson
Pos (0.9999999999999999, 0.0)
Neg (1.0, 0.0)
Pos/Neg (-0.999999999999956, 0.0)
Neg/Pos (-0.999999999999956, 0.0)


In [35]:
print('Kendall')
x = [i[0] for i in file['std answer']]
y = [i[0] for i in file['std answer']]
print('Pos',scipy.stats.kendalltau(x, y))

x = [i[1] for i in file['std answer']]
y = [i[1] for i in file['std answer']]
print('Neg',scipy.stats.kendalltau(x, y))

x = [i[0] for i in file['std answer']]
y = [i[1] for i in file['std answer']]
print('Pos/Neg',scipy.stats.kendalltau(x, y))

x = [i[1] for i in file['std answer']]
y = [i[0] for i in file['std answer']]
print('Neg/Pos',scipy.stats.kendalltau(x, y))

Kendall
Pos KendalltauResult(correlation=0.9999999999999998, pvalue=0.0)
Neg KendalltauResult(correlation=0.9999999999999998, pvalue=0.0)
Pos/Neg KendalltauResult(correlation=-0.9999999999999998, pvalue=0.0)
Neg/Pos KendalltauResult(correlation=-0.9999999999999998, pvalue=0.0)


In [34]:
print('Spearmna')
x = [i[0] for i in file['std answer']]
y = [i[0] for i in file['std answer']]
print('Pos',scipy.stats.spearmanr(x, y))

x = [i[1] for i in file['std answer']]
y = [i[1] for i in file['std answer']]
print('Neg',scipy.stats.spearmanr(x, y))

x = [i[0] for i in file['std answer']]
y = [i[1] for i in file['std answer']]
print('Pos/Neg',scipy.stats.spearmanr(x, y))

x = [i[1] for i in file['std answer']]
y = [i[0] for i in file['std answer']]
print('Neg/Pos',scipy.stats.spearmanr(x, y))

Spearmna
Pos SpearmanrResult(correlation=1.0, pvalue=0.0)
Neg SpearmanrResult(correlation=1.0, pvalue=0.0)
Pos/Neg SpearmanrResult(correlation=-1.0, pvalue=0.0)
Neg/Pos SpearmanrResult(correlation=-1.0, pvalue=0.0)


## Inconsistency sentences

In [24]:
count = 0
inconsistent_index = []
assert len(file['std answer']) == len(file['neg answer'])
for index, (pos,neg) in enumerate(zip(file['std answer'],file['neg answer'])):
    if pos.index(max(pos)) == neg.index(max(neg)):
        count +=1 
    else :
        inconsistent_index.append(index)
print('Obtained consistency', count/len(file['std answer']))

Obtained consistency 0.9558823529411765


In [27]:
for index_ in inconsistent_index:
    print('Neg prompt')
    print(file['neg prompt'][index_])
    print('Answer', file['neg answer'][index_].index(max(file['neg answer'][index_])) )
    print('Pos prompt')
    print(file['std prompt'][index_])
    print('Answer', file['std answer'][index_].index(max(file['std answer'][index_])) )
    print('---------------------------------------')

Neg prompt
Sentence 1: The results appear in the January issue of Cancer , an American Cancer Society journal , being published online today .
Sentence 2: The results appear in the January issue of Cancer , an American Cancer Society ( news - web sites ) journal , being published online Monday .
Do Sentence 1 and Sentence 2 express a different meaning? Yes or No?
Answer 1
Pos prompt
Sentence 1: The results appear in the January issue of Cancer , an American Cancer Society journal , being published online today .
Sentence 2: The results appear in the January issue of Cancer , an American Cancer Society ( news - web sites ) journal , being published online Monday .
Do Sentence 1 and Sentence 2 convey the same meaning? Yes or No?
Answer 0
---------------------------------------
Neg prompt
Sentence 1: Unable to find a home for him , a judge told mental health authorities they needed to find supervised housing and treatment for DeVries somewhere in California .
Sentence 2: The judge had tol